In [4]:
# !python -m pip install natsort

In [26]:
from pydub import AudioSegment
import os
from natsort import natsorted
from pydub import AudioSegment,silence
from pydub import AudioSegment, effects  
import datetime

### concat data

In [10]:
dataset="silence_segments"
a=os.listdir(dataset)
l=len(a)
print(l)

x=natsorted(a)
x
path=os.getcwd()
print(path)

os.makedirs("concat", exist_ok=True)

os.chdir(f"{path}/{dataset}")
print(os.getcwd())

for i in range(l):
    print(i)
    if i==0:
        if (x[i].endswith('.mp3')):
            sound1 = AudioSegment.from_mp3(x[i])
        else:
            sound1=AudioSegment.from_wav(x[i])
        continue
    else:
        if (x[i].endswith('.mp3')):
            sound2 = AudioSegment.from_mp3(x[i])
        else:
            sound2=AudioSegment.from_wav(x[i])

        combined_sounds=sound1+sound2
        sound1=combined_sounds

os.chdir(f'{path}')
print(os.getcwd())

combined_sounds.export(f'concat/sil_.wav', format="wav")

3
/home/anush/Desktop/ac_new
/home/anush/Desktop/ac_new/silence_segments
0
1
2
/home/anush/Desktop/ac_new


<_io.BufferedRandom name='concat/sil_xx.wav'>

### remove silence

In [2]:
!python -m pip install pyAudioAnalysis

In [6]:
%cd speech

/home/anush/Desktop/audio_classification/exp/speech


In [4]:
!python -m pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 600.0 kB/s eta 0:00:00m eta 0:00:010:00:01


In [8]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioSegmentation
import os
import soundfile as sf


# Input audio file (replace with your file path)
audio_file = "s1.wav"

# Output folder (replace with your desired output folder)
output_folder = "sil"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the audio
[rate, signal] = audioBasicIO.read_audio_file(audio_file)

# Perform silence removal
segments = audioSegmentation.silence_removal(signal, rate, 0.05, 0.02, smooth_window=1.0, weight=0.3, plot=False)

# Save the segments without silence to individual files
for i, segment in enumerate(segments):
    segment_path = os.path.join(output_folder, f"segment_{i}.wav")
    sf.write(segment_path, segment, rate)

print(f"{len(segments)} segments without silence saved in {output_folder}")

31 segments without silence saved in sil


### collect silence from concat file

In [23]:
file_list = os.listdir("data")
sort_files=natsorted(file_list)
print(len(sort_files))

print(sort_files)

26
['4175_8k.wav', '4484_8k.wav', '4485_8k.wav', '4490_8k.wav', '4504_8k.wav', '4520_8k.wav', '4521_8k.wav', '4522_8k.wav', '4537_8k.wav', '4543_8k.wav', '4544_8k.wav', '4547_8k.wav', '4708_8k.wav', '4745_8k.wav', '4823_8k.wav', '4874_8k.wav', '4889_8k.wav', '4984_8k.wav', '5000_8k.wav', '5051_8k.wav', '6269_8k.wav', '6285_8k.wav', '6526_8k.wav', '6561_8k.wav', '6661_8k.wav', '6750_8k.wav']


In [29]:
c=0
for filename in sort_files:
    c+=1
    if os.path.isfile(os.path.join("data", filename)):
        myaudio = AudioSegment.from_file(os.path.join("data", filename))
        myaudio = myaudio.set_frame_rate(8000)

        dBFS=myaudio.dBFS
        sil = silence.detect_silence(myaudio, min_silence_len=2000, silence_thresh=dBFS-28)
        
        si = [((start/1000),(stop/1000)) for start,stop in sil]
        print(filename, si)
        
        time_intervals = si
        output_dir="silence_segments"
        os.makedirs(output_dir, exist_ok=True)
        
        for i, (start_time, end_time) in enumerate(time_intervals):
            start_ms = int(start_time * 1000)
            end_ms = int(end_time * 1000)
            
            
            segment = myaudio[start_ms:end_ms]
            segment.export(f"{output_dir}/sil_28_2/silence_segment_28dbfs_{i}.wav", format="wav")

print(c)

4175_8k.wav [(1088.979, 1091.118)]
4484_8k.wav [(203.055, 205.535)]
4485_8k.wav []
4490_8k.wav [(261.262, 263.349), (350.825, 354.389), (904.904, 907.042), (1195.904, 1200.515), (1491.82, 1495.675), (1675.577, 1682.468)]
4504_8k.wav [(153.176, 155.197), (185.877, 190.354), (191.369, 198.111), (200.691, 204.678), (204.748, 215.29), (215.734, 221.118), (221.695, 223.9), (224.423, 226.826), (227.195, 229.354), (237.987, 240.064), (250.128, 252.551), (358.516, 362.673)]
4520_8k.wav [(24.526, 28.564)]
4521_8k.wav [(0.0, 2.179), (131.928, 133.951), (154.721, 156.872), (168.203, 170.289), (191.01, 194.831), (196.707, 199.361), (480.257, 482.972), (525.517, 529.957), (821.457, 827.216), (1598.631, 1600.739), (1601.365, 1603.637)]
4522_8k.wav [(564.839, 567.092), (712.698, 714.973), (894.459, 896.668), (1442.482, 1446.039), (1750.054, 1752.32)]
4537_8k.wav []
4543_8k.wav []
4544_8k.wav [(730.474, 732.503)]
4547_8k.wav []
4708_8k.wav [(414.655, 417.075), (821.521, 825.419), (832.793, 837.689), (

### downsample

In [15]:
import librosa
import soundfile as sf
import numpy as np

file_ls = os.listdir("data")
sort_file=natsorted(file_ls)
print(sort_file)

for filename in sort_file:
    print(filename)
    f=filename.split(".")
    print(f)
    a=librosa.load(os.path.join("data", filename), sr=8000)
    sf.write(f"silence_segments/{f[0]}_8k.wav", a[0], 8000)

### volume normalization

In [ ]:
file_list = os.listdir("normalize/d")
sort_files=natsorted(file_list)

for filename in sort_files:
    print(filename)
    f=filename.split(".")
    if os.path.isfile(os.path.join("normalize/d", filename)):
        s=datetime.datetime.now()
        rawsound = AudioSegment.from_file(os.path.join("normalize/d", filename), "wav")
        normalizedsound = effects.normalize(rawsound, headroom=0.05)
        normalizedsound.export(f"normalize/dn/{f[0]}_n.wav", format="wav")
        e=datetime.datetime.now()
        print("time: ", e-s)

In [27]:
rawsound = AudioSegment.from_file("n.wav", "wav")
normalizedsound = effects.normalize(rawsound, headroom=0.05)
normalizedsound.export(f"nn.wav", format="wav")

<_io.BufferedRandom name='nn.wav'>

In [ ]:
path='data/dataset_1sec'
for i in sorted(os.listdir(path)):
    os.makedirs(f"{path}/{i}_n", exist_ok=True)

    for j in os.listdir(os.path.join(path, i)):
        fn=j.split("/")
        fn=fn[-1].split(".")
        rawsound = AudioSegment.from_file(os.path.join(path, i, j), "wav")
        normalizedsound = effects.normalize(rawsound)
        normalizedsound.export(f"{path}/{i}_n/{fn[0]}.wav", format="wav")